# Iterative Quantum Amplitude Estimation (IQAE) module

The present notebook reviews the **Iterative Quantum Amplitude Estimation (IQAE)** algorithm.  

**BE AWARE**: This algorithm is different from the **Iterative Quantum Phase Estimation (IQPE)**. The latter is an algorithm for pure *phase estimation* of a unitary operator, while the former is an algorithm for directly solving the **Amplitude Estimation** problem based on the *amplification* capabilities of a Grover operator.  

The **IQAE** algorithm has been implemented in the module *iterative_quantum_ae* of the package **AE** in the library `QQuantLib` (**QQuantLib/AE/iterative_quantum_ae**). This algorithm is encapsulated in a Python class called `IQAE`.  

The present notebook and modules are based on the following references:  

- **Grinko, D., Gacon, J., Zoufal, C., & Woerner, S.**  
  Iterative Quantum Amplitude Estimation. npj Quantum Information **7**, (2021).  
  [https://www.nature.com/articles/s41534-021-00379-1](https://www.nature.com/articles/s41534-021-00379-1)  

- NEASQC deliverable:  
  *D5.1: Review of state-of-the-art for Pricing and Computation of VaR*  
  [https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf](https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]
linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int, measure_state_probability

## 1. Oracle generation

Before performing any amplitude estimation, we first need to load data into the quantum circuit. As this step is auxiliary and intended to demonstrate how the algorithm works, we will simply load a discrete probability distribution. 

In this example, we will use a quantum circuit with $ n = 3 $ qubits, which corresponds to a total of $ N = 2^n = 8 $ computational basis states. The discrete probability distribution we aim to load is defined as:

$$
p_d = \frac{(0, 1, 2, 3, 4, 5, 6, 7)}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}.
$$

This distribution assigns probabilities proportional to the integers $ 0 $ through $ 7 $, normalized by their sum to ensure that the total probability equals 1.

In [ ]:
n = 3
x = np.arange(2**n)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function `load_probability` from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. IQAE Algorithm

The problem of amplitude estimation can be stated as follows. Given an oracle operator $\mathcal{A}$:
$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, the goal is to estimate $\sqrt{a}$. We can define an associated angle $\theta$ such that $\sin^2{\theta} = a$, rewriting the problem as:
$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle. \tag{1}
$$

The foundation of any amplitude estimation algorithm is the Grover-like operator $\mathcal{Q}$ derived from the oracle operator $\mathcal{A}$:
$$
\mathcal{Q}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^\dagger \left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right).
$$
This Grover-like operator has the following effect on the state $|\Psi\rangle$:
$$
\mathcal{Q}^{m_k}|\Psi\rangle = \mathcal{Q}^{m_k} \mathcal{A} |0\rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle + \cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,
$$
where $m_k$ is an integer parameter.

Using these ingredients, the **IQAE** algorithm, given an input error $\epsilon$ and a confidence interval $\alpha$, allows us to estimate $(\theta_l, \theta_u)$ such that the angle $\theta$ of the **Amplitude Estimation (AE)** problem satisfies:
$$
P\big[\theta \in [\theta_l, \theta_u]\big] > 1 - \alpha
$$
and
$$
\frac{\theta_u - \theta_l}{2} \leq \epsilon.
$$

This result can be directly extended to $a = \sin^2{\theta}$, so the **IQAE** algorithm provides $(a_l, a_u)$ that satisfies:
$$
P\big[a \in [a_l, a_u]\big] > 1 - \alpha
$$
and
$$
\frac{a_u - a_l}{2} \leq \epsilon.
$$

### 2.1 The `IQAE` Class

We have implemented a Python class called `IQAE` in the **QQuantLib/AE/iterative_quantum_ae** module, which allows us to use the **IQAE** algorithm.

When creating the `IQAE` class, we followed the conventions used in the `MLAE` class from the **QQuantLib/AE/maximum_likelihood_ae** module. The following are the mandatory inputs for initializing the `IQAE` class:

1. `oracle`: A QLM `AbstractGate` or `QRoutine` that implements the Oracle for constructing the Grover operator.
2. `target`: The marked state in binary representation, provided as a Python list.
3. `index`: A list of qubits affected by the Grover operator.

Additionally, there are optional inputs for configuring the algorithm, which can be provided as a Python dictionary:
- `qpu`: The QLM solver to be used (default: PyLinalg).
- `epsilon` ($\epsilon$): The desired precision. Ensures that the width of the interval is at most $2\epsilon$ (default: 0.01).
- `alpha` ($\alpha$): The confidence level. Ensures that the probability of $a$ not lying within the given interval is at most $\alpha$ (default: 0.05).
- `shots`: The number of shots for each iteration of the algorithm (default: 100).
- `mcz_qlm`: A flag to use the QLM multi-controlled Z gate (`True`, default) or a multiplexor implementation (`False`).

---

#### Example

To demonstrate how the `IQAE` class and the algorithm work, consider the following amplitude estimation problem:

$$
|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right]. \tag{2}
$$

By comparing Equation (2) with Equation (1):

$$
\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle,
$$

and

$$
\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}} \left[\sqrt{0}|0\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

In this case, the target state is $|1\rangle$, whose binary representation is $001$. This must be passed to the `target` variable as a list (`[0, 0, 1]`). Additionally, we need to provide the list of qubits where the operation is being performed. In this example, it is $[0, 1, 2]$, corresponding to the entire register.

This setup allows us to estimate the amplitude $\sqrt{a}$ using the **IQAE** algorithm.

In [ ]:
#import the class
from QQuantLib.AE.iterative_quantum_ae import IQAE

In [ ]:
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

print('Real Value of a: ', a)

epsilon = 0.001
shots = 100
alpha = 0.05


In [ ]:
iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu,
    'mcz_qlm': True    
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

When the class is created the based oracle Grover operator is created too and can be acces using the `_grover_oracle`  property of the class

In [ ]:
c=iqae._grover_oracle
%qatdisplay c --svg --depth 2

### 2.2 IQAE Algorithm Scheme

As explained, the inputs for the **IQAE** algorithm are:
- Error in the estimation of the angle $\theta$: $\epsilon$.
- Confidence interval for $\theta$: $\alpha$.

The main steps of the **IQAE** algorithm, in a simplified form, are as follows:

1. **Initialization**: The algorithm initializes the limits for the angle to be estimated, $\theta$, to $[\theta_l, \theta_u] = [0, \frac{\pi}{2}]$.

2. **Calculation of Maximum Iterations**: The algorithm calculates the maximum number of iterations $T$ required to satisfy the error estimation $\epsilon$:
   $$
   T(\epsilon) \in \mathbb{N} \; / \; T(\epsilon) \geq \log_2\left(\frac{\pi}{8\epsilon}\right).
   $$
   In the context of the **IQAE** algorithm, an iteration corresponds to the selection of a different integer $k$.

3. **Selection of $k$**: This is the critical routine of the algorithm. The routine attempts to find the largest $k$ (up to a fixed limit) such that $(4k+2)\theta_l$ and $(4k+2)\theta_u$ are entirely contained within either the $[0, \pi]$ or $[\pi, 2\pi]$ semi-plane. If this condition is met, the selection routine returns the $k$ and the corresponding semi-plane.

4. **Circuit Creation for Selected $k$**: For the selected $k$, the **IQAE** algorithm creates the corresponding quantum circuit to perform:
   $$
   \mathcal{G}^{m}|\Psi\rangle = |\Psi\rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle + \cos\left((2m_k+1)\theta\right)|\Psi_1\rangle.
   $$

5. **Probability Estimation**: Using $N$ shots, compute the probability $a_k$ of obtaining $|\Psi_0\rangle$, which is given by:
   $$
   P(|\Psi_0\rangle, k) = \sin^2((2k+1)\theta) = \frac{1 - \cos((4k+2)\theta)}{2} = a_k.
   $$

6. **Error Calculation**: Using the number of measurements $N$, $T$, and $\alpha$, the algorithm calculates $\epsilon_{a_k}$ using:
   $$
   \epsilon_{a_k} = \sqrt{\frac{1}{2N} \log\left(\frac{2T}{\alpha}\right)}.
   $$

7. **Computation of Limits for $a_k$**: Using $\epsilon_{a_k}$, the algorithm computes the limits for $a_k$: $a_k^{\text{min}}$ and $a_k^{\text{max}}$.

8. **Computation of $\theta_k^{\text{min}}$ and $\theta_k^{\text{max}}$**: The algorithm computes $\theta_k^{\text{min}}$ and $\theta_k^{\text{max}}$ from $a_k^{\text{min}}$ and $a_k^{\text{max}}$, using:
   $$
   a_k = \frac{1 - \cos((4k+2)\theta)}{2},
   $$
   and the fact that $a_k^{\text{min}}$ and $a_k^{\text{max}}$ lie in one of the semi-planes: $[0, \pi]$ or $[\pi, 2\pi]$ (as determined by the selection routine in step 3).

9. **Updating $\theta_l$ and $\theta_u$**: The algorithm updates $\theta_l$ and $\theta_u$ using $\theta_k^{\text{min}}$ and $\theta_k^{\text{max}}$, respectively, and the fact that the rotation due to $k$ applications of the Grover operator is $(4k+2)\theta$.

At the end of each iteration, $\theta_u - \theta_l$ becomes smaller than at the beginning. The algorithm stops when $\theta_u - \theta_l \leq 2\epsilon$.

---

**NOTE**:
1. To ensure that $\theta_u - \theta_l \leq 2\epsilon$, the number of iterations should not exceed $T$, where:
   $$
   T(\epsilon) \geq \log_2\left(\frac{\pi}{8\epsilon}\right).
   $$

2. To ensure that $P\big[\theta \in [\theta_l, \theta_u]\big] > 1 - \alpha$, it is mandatory that the error of each iteration satisfies:
   $$
   \epsilon_{a_k} = \sqrt{\frac{1}{2N} \log\left(\frac{2T}{\alpha}\right)}.
   $$

### 2.3 Example of IQAE Workflow

Section 2.3 provides a simple illustration of the **IQAE** algorithm scheme. In this section, we present an example to provide intuition about how the **IQAE** algorithm operates. We will divide the algorithm into three main steps:

- **Initialization**.
- **First Iteration with $k = 0$**.
- **Subsequent Iterations with $k \geq 1$**.

#### 2.3.1 Initialization

We need to do the initialization of the algorithm (setting the initial $\theta_l$, $\theta_u$) and getting the maximum number of iterations from $\epsilon$ ($T(\epsilon)$)

In [ ]:
#Initialization of IQAE

[theta_l,theta_u] = [0.0,np.pi/2]
k=0
#True for semiplane [0,pi]
flag = True
#Number of shots
shots = 100
#Number of iterations
T = int(np.ceil(np.log2(np.pi/(8*iqae.epsilon)))+1)

print('Max number of IQAE iterations: ',T)


#### 2.3.2 First iteration with $k=0$.

In the first iteration, we are going to set $k=0$. Then we execute the complete iteration workflow:

In [ ]:
#First step
N=shots
print('#################### First Iteration k= {}. Start #################'.format(k))
print('k = ', k)
K = 4*k+2
print('K = 4*k+2= ', K)
DeltaTheta_initial = np.abs(theta_u-theta_l)
print('Creating the Quantum circuit with k= ', k)
routine = qlm.QRoutine()
wires = routine.new_wires(iqae.oracle.arity)
routine.apply(iqae.oracle,wires)
for j in range(k):
    routine.apply(iqae._grover_oracle,wires)
    
print('Computing the probabiliy of measure |Phi_0>')
results,_,_,_, = get_results(
    routine,linalg_qpu = linalg_qpu,
    shots = 10,
    qubits = iqae.index
)
#Probability of measure |Phi_0>

a = measure_state_probability(results, iqae.target)
print('probability of measure |Phi_0> for {}: {} (a)'.format(k, a))
#Getting the error for a
epsilon_a = iqae.chebysev_bound(N,alpha/T)
print('epsilon for iteration {}: {}'.format(k, epsilon_a))
#using epsilon we compute new a limits
a_max = np.minimum(a+epsilon_a,1.0)
a_min = np.maximum(a-epsilon_a,0.0)
#getting theta_min and theta_min from a_min,a_max
[theta_min,theta_max] = iqae.invert_sector(a_min,a_max,flag)
#Updating theta_l and theta_u from theta_min,theta_max and K
theta_l = (2*np.pi*np.floor(K*theta_l/(2*np.pi))+theta_min)/K
theta_u = (2*np.pi*np.floor(K*theta_u/(2*np.pi))+theta_max)/K
print('New: [theta_l, theta_u]= [{}, {}]'.format(theta_l, theta_u))
DeltaTheta_present = np.abs(theta_u-theta_l)
print('#################### First Iteration k= {}. End #################'.format(k))

In [ ]:
c = routine.to_circ()
%qatdisplay c --depth 0 --svg

Now we compare the difference between the olds and the new $\theta_u$ and $\theta_l$

In [ ]:
print('Initial Delta Theta: ', DeltaTheta_initial)
print('Final Delta Theta: ', DeltaTheta_present)

As can be seen the difference now is lower

#### 2.3.3 Next Iterations with $ k \geq 0 $

In the subsequent iterations, the first step is to determine the $ k $ for the current iteration. As explained in Step 3 of Section 2.2, this is the **critical** routine of the algorithm. This **routine** uses the current values of $ \theta_l $, $ \theta_u $, and the $ k $ from the previous iteration to compute the largest $ k $ (up to a predefined limit) that ensures $ (4k+2)\theta_l $ and $ (4k+2)\theta_u $ are entirely contained within either the $ [0, \pi] $ or $ [\pi, 2\pi] $ semi-plane.

This is achieved using the `find_next_k` method of the class. The method requires the following inputs:
- `k`: The $ k $ value from the previous iteration.
- `theta_lower`: The current lower bound $ \theta_l $.
- `theta_upper`: The current upper bound $ \theta_u $.
- `flag`: A flag for tracking the semi-plane (True for $ [0, \pi] $).
- `r`: A parameter of the routine (default value is 2).

The outputs of the method will be:
- `k`: The new $ k $ for the current iteration.
- `flag`: The semi-plane where $ (4k+2)\theta_l $ and $ (4k+2)\theta_u $ will be contained (True for $ [0, \pi] $).

To execute the complete iteration, run the following cell.

In [ ]:
print('Searching for the new k')
[k,flag] = iqae.find_next_k(k,theta_l,theta_u,flag)

print('#################### ITERATION with k = {}. Start #################'.format(k))
print('New k= ', k)
K = 4*k+2
print('New K= 4*k+2= ', K)

DeltaTheta_initial = np.abs(theta_u-theta_l)

print('Creating the Quantum circuit with k= ', k)
routine = qlm.QRoutine()
wires = routine.new_wires(iqae.oracle.arity)
routine.apply(iqae.oracle,wires)
for j in range(k):
    routine.apply(iqae._grover_oracle,wires)
    
print('Computing the probability of measure |Phi_0>')    
results,_,_,_ = get_results(
    routine,linalg_qpu = linalg_qpu,
    shots = N,
    qubits = iqae.index
)
#Probability of measure |Phi_0>
#a = results['Probability'].iloc[bitfield_to_int(iqae.target)]
a = measure_state_probability(results, iqae.target)
print('probability of measure |Phi_0> for {}: {} (a)'.format(k, a))

#Getting the error for a
epsilon_a = iqae.chebysev_bound(N,alpha/T)
print('epsilon for iteration {}: {}'.format(k, epsilon_a))
#using epsilon we compute new a limits
a_max = np.minimum(a+epsilon_a,1.0)
a_min = np.maximum(a-epsilon_a,0.0)
#getting theta_min and theta_min from a_min,a_max
[theta_min,theta_max] = iqae.invert_sector(a_min,a_max,flag)
#Updating theta_l and theta_u from theta_min,theta_max and K
theta_l = (2*np.pi*np.floor(K*theta_l/(2*np.pi))+theta_min)/K
theta_u = (2*np.pi*np.floor(K*theta_u/(2*np.pi))+theta_max)/K
print('New: [theta_l, theta_u]= [{}, {}]'.format(theta_l, theta_u))
DeltaTheta_present = np.abs(theta_u-theta_l)

print('#################### ITERATION with k = {}. End #################'.format(k))
print('Initial Delta Theta: ', DeltaTheta_initial)
print('Final Delta Theta: ', DeltaTheta_present)

In [ ]:
c = routine.to_circ()
%qatdisplay c --depth 0 --svg

In order to do several iterations execute the cell several times.

In [ ]:
print('Is enough: ', DeltaTheta_present < iqae.epsilon)

Sometimes the routine for finding the new $k$ cannot get a proper new $k$, then the old $k$ is used again. To avoid repeat the same $k$ a lot of times we can accumulate the measurements done for one $k$ and use them for calculating the step error $\epsilon_{a_{k}}$

## 3. IQAE Complete Execution

In Section 2.4, the basic scheme of the **IQAE** algorithm was outlined for pedagogical purposes. The `IQAE` class encapsulates the code presented in Section 2.4 (while implementing additional optimizations for better performance) in a user-friendly manner. It is expected that users of the `IQAE` class will primarily interact with the following methods:
- `iqae` method
- `run` method
- `display_information` method

### 3.1 The `iqae` Method

To execute the complete algorithm using the `IQAE` class, the `iqae` method is used. This method accepts the following inputs:
- `epsilon` ($\epsilon$): Error in the estimation of the angle $\theta$ (default: 0.01).
- `shots`: Number of shots for the measurement of the circuit ($N_{\text{shots}}$) (default: 100).
- `alpha` ($\alpha$): Confidence interval for $\theta$ (default: 0.05).

This method returns the limits for the $a$ estimation: $(a_{\min}, a_{\max})$.

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

epsilon = 0.001
shots = 100
alpha = 0.05

iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu,
    'mcz_qlm': True    
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

In [ ]:
epsilon_t = 0.001
[a_l, a_u]=iqae.iqae(
    epsilon = epsilon_t,
    shots = 500,
    alpha=0.01
)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(a_l, a_u))
a_estimated = (a_u+a_l)/2.0
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a))
print('Error estimation wanted: ', epsilon_t)

We can obtain the complete statistics of all the circuits used during the algorithm execution calling the `circuit_statistics` attribute

In [ ]:
iqae.circuit_statistics

### 3.2 `display_information` method

The `display_information` method gives some information of the inner working of the **IQAE** algorithm. The inputs are the same that for the *iqae* method.

In [ ]:
iqae.display_information(
    epsilon = iqae.epsilon,
    shots= iqae.shots, 
    alpha = iqae.alpha
)

### 3.3 The `run` Method

A `run` method has been implemented for the direct execution of the **IQAE** algorithm. In this case, the user can configure all the properties of the `IQAE` class, and the `run` method will execute the algorithm using the predefined attributes of the class. 

The method returns the estimation of $ a = \frac{a_u + a_l}{2} $. Additionally, the `run` method populates the following class attributes:

- `ae_l`: The lower limit for $ a $: $ a_l $.
- `ae_u`: The upper limit for $ a $: $ a_u $.
- `theta_l`: The lower limit for $ \theta $: $ \theta_l $.
- `theta_u`: The upper limit for $ \theta $: $ \theta_u $.
- `ae`: The amplitude estimation parameter, computed as $ a = \frac{a_u + a_l}{2} $.
- `theta`: The estimated angle $ \theta = \frac{\theta_u + \theta_l}{2} $.
- `run_time`: The elapsed time for the execution of the `run` method.

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

epsilon = 0.001
shots = 100
alpha = 0.05

iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu,
    'mcz_qlm': True       
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

In [ ]:
a_estimated = iqae.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a)
print('Bounds for a: [iqae.ae_l, iqae.ae_u] = [{}, {}]'.format(iqae.ae_l, iqae.ae_u))
print('Bounds for theta: [iqae.theta_l, iqae.theta_u] = [{}, {}]'.format(iqae.theta_l, iqae.theta_u))
print('Estimated theta: iqae.theta = ', iqae.theta)
print('Estimated a: iqae.ae = ', iqae.ae)

In [ ]:
print(' a_real-iqae.ae: ', abs(iqae.ae-a))
print('Epsilon: ', iqae.epsilon)
print('iqae error: ', iqae.ae_u-iqae.ae_l)

In [ ]:
print("Elapsed time for the run method: ", iqae.run_time)

In [ ]:
c = iqae._grover_oracle
%qatdisplay c --depth 3 --svg

When the *run* method is executed following class attributes are populated:

* `circuit_statistics`: Python dictionary with the statistics of each circuit used during the algorithm execution. Each key of the dictionary corresponds with a $k$ application of the Grover-like operator used and its associated value is a Python dictionary with the complete statistical information of the circuit created for each $k$ value.
* `schedule_pdf`: pandas DataFrame with the complete schedule used in the algorithm execution. The schedule lists the number of applications Grover-like applications and the number of shots used for measurements.
* `oracle_calls`: number of total oracle calls for a complete execution of the algorithm.
* `max_oracle_depth`: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
#Print gates statistic of each used circuit
iqae.circuit_statistics

In [ ]:
iqae.schedule_pdf

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the IQAE was: {}".format(iqae.oracle_calls))

In [ ]:
#Number of maximum oracle applications
iqae.max_oracle_depth

In [ ]:
iqae.quantum_times

In [ ]:
iqae.quantum_time

In [ ]:
iqae.run_time

## 4. Modified IQAE

The modified Iterative Quantum Amplitude Estimation (**mIQAE**) algorithm represents an improvement over the **IQAE** algorithm, offering enhanced performance. The **mIQAE** algorithm is described in the following paper:

- **Fukuzawa, Shion**, **Ho, Christopher**, **Irani, Sandy**, and **Zion, Jasen**: *Modified Iterative Quantum Amplitude Estimation is Asymptotically Optimal*. 2023 Proceedings of the Symposium on Algorithm Engineering and Experiments (ALENEX). Society for Industrial and Applied Mathematics.

The primary contribution of the **mIQAE** algorithm is its ability to adapt the probability of failure, $\alpha_i$, and the corresponding number of shots at each step of the algorithm. In contrast, the failure probability in the **IQAE** algorithm remains constant across all steps. This modification allows the **mIQAE** algorithm to achieve superior query performance compared to the original **IQAE**:

- **IQAE** query complexity: $\sim \frac{1}{\epsilon} \log \left( \frac{1}{\alpha} \log \left(\frac{1}{\epsilon}\right)\right)$
- **mIQAE** query complexity: $\sim \frac{1}{\epsilon} \log \frac{1}{\alpha}$

The **mIQAE** algorithm has been implemented in the **QQuantLib.AE** package, specifically in the module *modified_iterative_quantum_ae*, within the class `mIQAE`. 

The functionality of the `mIQAE` class is identical to that of the `IQAE` class, ensuring a seamless transition for users familiar with the original implementation.

In [ ]:
from QQuantLib.AE.modified_iterative_quantum_ae import mIQAE

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

epsilon = 0.001
shots = 100
alpha = 0.05

miqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu,
    'mcz_qlm': True       
}

miqae = mIQAE(oracle, target = target, index = [0,1,2], **miqae_dict)

In [ ]:
# Run method works in the same way than IQAE class
a_estimated_miqae = miqae.run()

In [ ]:
print('a_estimated: ', a_estimated_miqae)
print('Real Value of a: ', a)
print('Bounds for a: [miqae.ae_l, miqae.ae_u] = [{}, {}]'.format(miqae.ae_l, miqae.ae_u))
print('Bounds for theta: [miqae.theta_l, miqae.theta_u] = [{}, {}]'.format(miqae.theta_l, miqae.theta_u))
print('Estimated theta: iqae.theta = ', miqae.theta)
print('Estimated a: iqae.ae = ', miqae.ae)

In [ ]:
print(' a_real-miqae.ae: ', abs(miqae.ae-a))
print('Epsilon: ', iqae.epsilon)
print('miqae error: ', miqae.ae_u-miqae.ae_l)

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the mIQAE was: {}".format(miqae.oracle_calls))

In [ ]:
# Comparing IQAE and mIQAE

print("IQAE estimation: {}. mIQAE estimation: {}".format(iqae.ae, miqae.ae))
print("IQAE error: {}. mIQAE erro: {}".format(abs(iqae.ae-a), abs(miqae.ae-a)))
print("Number of Oracle Calls: IQAE: {}. mIQAE: {}".format(iqae.oracle_calls, miqae.oracle_calls))

We can compare the bounds for both methods by calling the method `compute_info` that provides info about the bounds onf the algorithm

In [ ]:
eps_list= np.logspace(-1, -7)
iqae_grover = [iqae.compute_info(x, shots=100, alpha=0.05)["n_oracle"] for x in eps_list]
miqae_grover = [miqae.compute_info(x, shots=100, alpha=0.05)["n_oracle"] for x in eps_list]

In [ ]:
plt.plot(eps_list, iqae_grover)
plt.plot(eps_list, miqae_grover)
plt.xscale("log")
plt.yscale("log")
xmin, xmax, ymin, ymax = plt.axis()
plt.xlim(xmax, xmin)
plt.legend(["IQAE", "mIQAE"])
plt.xlabel(r"$\epsilon$")
plt.ylabel(r"Oracle Calls")